## To Do

- what is a good chunk size?
- how can I verify a better performance?
- check accuracy of results
- check computation time on cluster

In [1]:
from toad import TOAD
import numpy as np
import xarray as xr
from toad.shifts_detection.methods import ASDETECT as ASDETECT
#import dask

fp = "tutorials/test_data/garbe_2020_antarctica.nc"
#fp = "tutorials/test_data/global_mean_summer_tas.nc"
var = "thk"

data = xr.open_dataset(fp)
spatial_dims = list(data[var].dims)
spatial_dims.remove("time")

c = 5
c_dict = {dim: c for dim in spatial_dims}
c_dict["time"] = 3
data = data.coarsen(**c_dict,
                    boundary="trim").reduce(np.mean)

print(f"Dimensions after coarsening:\n{data.dims}")


Dimensions after coarsening:
FrozenMappingWarningOnValuesAccess({'time': 116, 'y': 38, 'x': 38})


In [6]:
from dask.distributed import Client

client = Client()
client


/home/lukasro/Documents/TIPMIP/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33901 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:33901/status,
Dashboard: http://127.0.0.1:33901/status,Workers: 4
Total threads: 8,Total memory: 15.29 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34845,Workers: 0
Dashboard: http://127.0.0.1:33901/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33603,Total threads: 2
Dashboard: http://127.0.0.1:34519/status,Memory: 3.82 GiB
Nanny: tcp://127.0.0.1:42257,


In [2]:
#dask.config.set(scheduler='threads')

td_new = TOAD(data)
td_new.compute_shifts(var,
                  method=ASDETECT(),
                  overwrite=True,
                  chunk_size=10,)

[########################################] | 100% Completed | 11.89 s


## Chunk Size

In [3]:
import time

# Define the chunk sizes to test
chunk_sizes = [None, 5, 20, 50]
sample_size = 5

# Run tests
results = [0] * len(chunk_sizes)
print(results)

print("Benchmarking chunk sizes...\n")
for i in range(len(chunk_sizes)):
    size = chunk_sizes[i]
    for j in range(sample_size):
        # get test data
        td = TOAD(data)

        # Time the execution
        start_time = time.time()
        td_new.compute_shifts(var,
                        method=ASDETECT(),
                        overwrite=True,
                        chunk_size=size,)
        elapsed = time.time() - start_time

        results[i] += elapsed
    results[i] /= sample_size
    print(f"Chunk size {size}x{size}: {results[i]:.2f} seconds")

# Summary
#print("\nSummary:")
#for size, elapsed in results:
#    print(f"Chunk size {size}x{size}: {elapsed:.2f} s")


[0, 0, 0, 0]
Benchmarking chunk sizes...

[                                        ] | 0% Completed | 222.38 us

[########################################] | 100% Completed | 1.59 sms
[########################################] | 100% Completed | 1.54 sms
[########################################] | 100% Completed | 1.52 sms
[########################################] | 100% Completed | 1.59 sms
[########################################] | 100% Completed | 1.55 sms
Chunk size NonexNone: 1.59 seconds
[########################################] | 100% Completed | 1.69 sms
[########################################] | 100% Completed | 1.69 sms
[########################################] | 100% Completed | 1.70 sms
[########################################] | 100% Completed | 1.69 sms
[########################################] | 100% Completed | 1.62 sms
Chunk size 5x5: 1.72 seconds
[########################################] | 100% Completed | 1.58 sms
[########################################] | 100% Completed | 1.58 sms
[########################################] | 100% Completed | 1.49 sms
[############

## Artificial Dataset

In [ ]:
import xarray as xr
import numpy as np

# Define large shape (e.g., 1000 time steps, 1000x1000 spatial grid)
shape = (500, 1000, 1000)  # (time, lat, lon)

# Create coordinate values
time = np.arange(shape[0])
lat = np.linspace(-90, 90, shape[1])
lon = np.linspace(-180, 180, shape[2])

# Create synthetic data using Dask
data = xr.DataArray(
    np.random.rand(*shape),
    dims=["time", "lat", "lon"],
    coords={"time": time, "lat": lat, "lon": lon}
)
td_art = TOAD(data) 

print(td_art.data.dims)


('time', 'lat', 'lon')


: 

In [ ]:
import time

# Define the chunk sizes to test
chunk_sizes = [50, 100, 500]

# Run tests
results = []

print("Benchmarking chunk sizes...\n")
for size in chunk_sizes:
    # get test data
    td = TOAD(data)

    # Time the execution
    start_time = time.time()
    td_art.compute_shifts(var,
                    method=ASDETECT(),
                    overwrite=True,
                    chunk_size=size,)
    elapsed = time.time() - start_time

    results.append((size, elapsed))
    print(f"Chunk size {size}x{size}: {elapsed:.2f} seconds")

# Summary
print("\nSummary:")
for size, elapsed in results:
    print(f"Chunk size {size}x{size}: {elapsed:.2f} s")


Benchmarking chunk sizes...

